In [16]:
!pip install ktrain
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

import ktrain
from ktrain import text
import numpy as np
import glob
import pandas as pd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-qnyblx98
  Running command git clone --filter=blob:none --quiet https://github.com/amaiya/eli5 /tmp/pip-req-build-qnyblx98
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfke

In [17]:
trainDf = pd.read_csv('../input/sentnob-sentiment-analysis-in-noisy-bangla-texts/SentNoB Dataset/Train.csv')
valDf = pd.read_csv('../input/sentnob-sentiment-analysis-in-noisy-bangla-texts/SentNoB Dataset/Val.csv')

In [18]:
valDf

,Data,Label
0,আর আমার খুবেই আনন্দ লাকছে ভাইটি চাকরি পেয়েছে,1
1,ভাই আমাদের আগের মেয়র আনিচুল হক নাই যে আমাদের ক...,2
2,আমি মার্ক ওয়েন আর সনির বিশাল ভক্ত । একটা সময় ভ...,1
3,৩ মাস না যেতেই একেকজন ফুলে ফেপে আলুর দম,2
4,"বাংলাদেশের পুলিশ হলো নিরীহ মানুষের যম , আর অত্...",2
...,...,...
1562,আর বিচার হবে বলে মনে হয় না,2
1563,"ভাই রাতে গেলে পাবেন , ০১৮৬২২৮৪৪৯৫ অথবা ০১৮৬২৮০...",1
1564,আমাকে যে দিন ( যেই তারিখ এ ) ডক্টর এ্যাপয়েনমে...,1
1565,আরে ভাই এখান থেকে তো রাজনীতি করে বড় হচ্ছে আর য...,2


In [19]:
from sklearn.model_selection import train_test_split
train_body, test_body, train_cluster_id, test_cluster_id = trainDf['Data'].values, valDf['Data'].values, trainDf['Label'].values, valDf['Label'].values

In [20]:
#for Bengali

MODEL_NAME = 'csebuetnlp/banglabert'  # 
# csebuetnlp/banglabert
t = text.Transformer(MODEL_NAME, maxlen=200)


In [21]:
X_train = list(train_body)
y_train = list(train_cluster_id)
X_test = list(test_body)
y_test = list(test_cluster_id)

y_train = [str(x) for x in train_cluster_id]
y_test = [str(x) for x in test_cluster_id]

trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 15
	95percentile : 37
	99percentile : 47


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 37
	99percentile : 47


In [31]:
# learning_rate = 1e-4
learning_rate = 0.001
epochs = 5

model = t.get_classifier()

learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=16)

learner.fit_onecycle(learning_rate, epochs)

404 Client Error: Not Found for url: https://huggingface.co/csebuetnlp/banglabert/resolve/main/tf_model.h5
/opt/conda/lib/python3.7/site-packages/ktrain/text/preprocessor.py:1172: UserWarning: Could not load a Tensorflow version of model. (If this worked before, it might be an out-of-memory issue.) Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  "Could not load a Tensorflow version of model. (If this worked before, it might be an out-of-memory issue.) "




begin training using onecycle policy with max lr of 0.001...
Epoch 1/5
786/786 [==============================] - 344s 416ms/step - loss: 1.0750 - accuracy: 0.4041 - val_loss: 1.0747 - val_accuracy: 0.3765
Epoch 2/5
786/786 [==============================] - 327s 413ms/step - loss: 1.0748 - accuracy: 0.3981 - val_loss: 1.0709 - val_accuracy: 0.3976
Epoch 3/5
786/786 [==============================] - 327s 412ms/step - loss: 1.0743 - accuracy: 0.4002 - val_loss: 1.0768 - val_accuracy: 0.3976
Epoch 4/5
786/786 [==============================] - 326s 412ms/step - loss: 1.0728 - accuracy: 0.4056 - val_loss: 1.0709 - val_accuracy: 0.3976
Epoch 5/5
786/786 [==============================] - 327s 413ms/step - loss: 1.0721 - accuracy: 0.4083 - val_loss: 1.0708 - val_accuracy: 0.3976


In [32]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       354
           1       0.40      1.00      0.57       623
           2       0.00      0.00      0.00       590

    accuracy                           0.40      1567
   macro avg       0.13      0.33      0.19      1567
weighted avg       0.16      0.40      0.23      1567



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[  0, 354,   0],
       [  0, 623,   0],
       [  0, 590,   0]])

In [33]:
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.save('combined_model')

In [34]:
learner.model.summary()

Model: "tf_electra_for_sequence_classification_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
electra (TFElectraMainLayer) multiple                  110026752 
_________________________________________________________________
classifier (TFElectraClassif multiple                  592899    
Total params: 110,619,651
Trainable params: 110,619,651
Non-trainable params: 0
_________________________________________________________________
